# Webscrap de Amazon:
### funcion que scrapea cada item, y funcion que scrapea un tiem y si este baja de precio te envia un mail

In [4]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import smtplib

In [5]:
URL = "https://www.amazon.com/s?k=messi&__mk_es_US=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2BGCYOGCOFVLN&sprefix=messi%2Caps%2C383&ref=nb_sb_noss_1"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

page = requests.get(URL, headers=headers)

soup = BeautifulSoup(page.content, "html.parser")

soup2 = BeautifulSoup(soup.prettify(), "html.parser")


In [6]:
page.status_code

200

In [7]:
secciones = soup.find_all("div", attrs = {"class":"a-section a-spacing-base"})


lista=[]
for seccion in secciones: 
    x= seccion.find("h2")
    lista.append(x)

In [8]:
links_secciones = [seccion.a.get("href") for seccion in lista]

In [9]:
links_secciones

['/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A0371377948QLNP2X8EO&qualifier=1676162328&id=224018740294532&widgetName=sp_atf&url=%2FLionel-Posters-Decoration-16x24inch-40x60cm%2Fdp%2FB0BQV8LBKL%2Fref%3Dsr_1_1_sspa%3F__mk_es_US%3D%25C3%2585M%25C3%2585%25C5%25BD%25C3%2595%25C3%2591%26crid%3D2BGCYOGCOFVLN%26keywords%3Dmessi%26qid%3D1676162328%26sprefix%3Dmessi%252Caps%252C383%26sr%3D8-1-spons%26psc%3D1',
 '/SoccerStarz-SOC1209-Argentina-Lionel-Figure/dp/B07BN3SQQK/ref=sr_1_2?__mk_es_US=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2BGCYOGCOFVLN&keywords=messi&qid=1676162328&sprefix=messi%2Caps%2C383&sr=8-2',
 '/MJGUYS-Youth-Sportswear-Soccer-Sleeve/dp/B0BK24BSJR/ref=sr_1_3?__mk_es_US=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2BGCYOGCOFVLN&keywords=messi&qid=1676162328&sprefix=messi%2Caps%2C383&sr=8-3',
 '/Sticker-Waterproof-Skateboard-Scrapbook-Thanksgiving/dp/B09KKC1PY8/ref=sr_1_4?__mk_es_US=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2BGCYOGCOFVLN&keywords=messi&qid=167616232

In [10]:
h2s = [seccion.find("span").text for seccion in lista]

In [11]:
print(len(h2s))
print(len(links_secciones))

66
66


In [52]:


def get_links_limpia(UERRELE):
    
    ### primero mandamos a BS el link 
    URL = UERRELE
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    page = requests.get(URL, headers=headers)

    soup = BeautifulSoup(page.content, "html.parser")

    soup2 = BeautifulSoup(soup.prettify(), "html.parser")
    
    ## Encontramos las secciones para extraer los hrefs de cada producto que aparece en la pagina
    
    secciones = soup.find_all("div", attrs = {"class":"a-section a-spacing-base"})

    ## extreamos los hrefs 
    
    lista=[]
    for seccion in secciones: 
        x= seccion.find("h2")
        lista.append(x)
    links_secciones = [seccion.a.get("href") for seccion in lista]
    
    print(links_secciones)
    ### Extraemos los h2s de cada producto en la pagina, para botar aquellos q petenezcan a un grupo de ROPA
    ### Aca quiero dejar de lado ropa por motivos de NLP en donde solo sean GOODIES
    
    h2s = [seccion.find("span").text for seccion in lista]
    
    ## Para ello creamos un diccionario para hacer un match de palabras y luego botar por indice todas las URLS q no sirvan
    
    diccionario_enngrishu = ["SHIRT", "shirt", "troussers", "TROUSSERS", "SOCKS","socks","jersey","JERSEY","cap","CAP", "jockey","JOCKEY"]
    diccionario_enngrishu_1 = []
    
    for x in diccionario_enngrishu:
        if x is not x.upper():
            xx = x[0].upper() + x[1:]
            diccionario_enngrishu_1.append(xx)
    diccionario_english_total = diccionario_enngrishu_1+diccionario_enngrishu
    
    ### Diccionario terminado
    
    ### En caso que el h2 contenga uan palabra del diccionario de ropa en ingles, entonces extraemos sus indices con la 
    ### vaina esta de abajo:    
    lista_indices = [] 
      
    for i, j in enumerate(h2s):
        for x in diccionario_english_total:
            if x in j:
                 lista_indices.append(i)

    sin_duplis = [*set(lista_indices)]
        
    ### ya que hemos extraido lpos indices, y las len de los h2 con las len de los Links secciones son giuales y ordenados de igual forma
    ## procedemos a botar todos esos indices que contengan ROPAS.
    
    ga = []    
    for idx, ele in enumerate(links_secciones):
         # chekeamos si el elemento no está presente en la lista de secciones links
        if idx not in sin_duplis:
            ga.append(ele)

            ### En caso las len sean diferentes significa que algo adna mal    
    if len(ga) != (len(links_secciones) - len(sin_duplis)):
        print("ERROR, algo pasa se están eliminando pocas o muchas secciones, revisa esta definicion")
            
    return ga

In [53]:
def itera(links):
    links = links
    for x in links:
        scrapea_url('https://www.amazon.com' + x)
        ### Aplicamos la definicion "scrapea_url()" para cada link sacado

In [54]:
def scrapea_url(URL):
          ### Tomamos la URL y aplicamos todo el proceso para usar BS y scrapear.
    URL = URL

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    page = requests.get(URL, headers=headers)

    soup1 = BeautifulSoup(page.content, "html.parser")

    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")
    
    time.sleep(0.7)
    
    ## Encontramos el precio, titulo, y reviews.
            
    if soup2.find("span", class_='a-offscreen') is not None:
        time.sleep(0.5)
        price = soup2.find("span", class_='a-offscreen').text
        time.sleep(0.7)
        price = price.strip()
        price = price[0:]
    else:
        price = None
    time.sleep(0.7)
    
    if soup2.find("span", id='productTitle') is not None:
        time.sleep(0.5)
        title = soup2.find("span", id='productTitle').text
        time.sleep(0.3)
        title = title.strip()
        time.sleep(0.7)
    else:
        title = None
    time.sleep(1.7)
    
    reviews = soup2.find_all("span", class_='cr-original-review-content')
    
    time.sleep(0.99)
    
    reviewss = []
    for r in reviews:
        rr = r.text
        time.sleep(0.9)
        rr = rr.strip()
        time.sleep(0.9)
        reviewss.append(rr)
      
    
    # importmaos datetime en caso querramos crear una ALARMA 
    
    import datetime
    
    today = datetime.date.today()
    
    
    ## Guardamos en un DF, puedes crear uno antes con un HEADER como el declarado abajo.
    import csv     
    
    header = ['Title', 'Price','Reviews','Date']
    data = [title, price, reviewss , today]

    with open('AmazonMessiScrapper.csv', 'a+', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)

#### PROBANDO EL SCRAPER

In [42]:
URL = "https://www.amazon.com/s?k=messi&__mk_es_US=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2BGCYOGCOFVLN&sprefix=messi%2Caps%2C383&ref=nb_sb_noss_1"
mis_links_scrap = get_links_limpia(URL)

['/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A0480507KSNV14Z3RX0S&qualifier=1676236300&id=7317857341681355&widgetName=sp_atf&url=%2FSignables-Premium-Facsimile-Memorabilia-Collectors%2Fdp%2FB09NWHRHSK%2Fref%3Dsr_1_1_sspa%3F__mk_es_US%3D%25C3%2585M%25C3%2585%25C5%25BD%25C3%2595%25C3%2591%26crid%3D2BGCYOGCOFVLN%26keywords%3Dmessi%26qid%3D1676236300%26sprefix%3Dmessi%252Caps%252C383%26sr%3D8-1-spons%26psc%3D1%26smid%3DA2ZLBRDQAAZOTP', '/Sticker-Waterproof-Skateboard-Scrapbook-Thanksgiving/dp/B09KKC1PY8/ref=sr_1_2?__mk_es_US=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2BGCYOGCOFVLN&keywords=messi&qid=1676236300&sprefix=messi%2Caps%2C383&sr=8-2', '/SoccerStarz-SOC1209-Argentina-Lionel-Figure/dp/B07BN3SQQK/ref=sr_1_3?__mk_es_US=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2BGCYOGCOFVLN&keywords=messi&qid=1676236300&sprefix=messi%2Caps%2C383&sr=8-3', '/COMTRUDE-Football-Pendant-Stainless-Necklace/dp/B08GCDM7WG/ref=sr_1_4?__mk_es_US=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2BGCYO

In [43]:
len(mis_links_scrap)

55

In [55]:
mis_links_scrap = mis_links_scrap[4:]

In [56]:
itera(mis_links_scrap)

In [ ]:
import pandas as pd

df1 = pd.read_csv("AmazonMessiScrapper.csv")

df1.tail(20)

## Case you want a particular item at a certain price & and sending an email to u

In [1]:
def scrapea_1_producto(URL):
          ### Tomamos la URL y aplicamos todo el proceso para usar BS y scrapear.
    URL = URL

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    page = requests.get(URL, headers=headers)

    soup1 = BeautifulSoup(page.content, "html.parser")

    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")
    
    time.sleep(0.7)
    
    ## Encontramos el precio, titulo, y reviews.
            

    price = soup2.find("span", class_='a-offscreen').text
    print(price)
    price = price.strip()

    print(price)

    ### los characetrs del price cambian dependiendo del precio del producto, lo ideal sería extraer 
    # caracteres con la funcion de abajo
    ### Pero en caso sea algo sencillo puedes usar los indices para quedarte solo con los numeros que quieras
    
    price = price[:]

    ## Tienes que eliminar las comas de los precios que tengan mas o igual de 1000 xq hay una COMA

    coma=[",","S","$","U"]
    res=[j for i,j in enumerate(price) if j not in coma]
    price = "".join(res)

    price = float(price)

    import datetime
    
    today = datetime.date.today()
    
    if price < 5:
        send_mail()
    


In [2]:
def send_mail():
    server = smtplib.SMTP_SSL('smtp.gmail.com',465)
    server.ehlo()
    #server.starttls()
    server.ehlo()
    server.login('msvf96@sa.com','xxxxxxxxxxxxxx')
    
    subject = "Tu Messi cabezaón está a menos de 2 dolares!"
    
    ### HAZ UN EXTRACT DEL URL
    body = "Alex, This is the moment we have been waiting for. Now is your chance to pick up the shirt of your dreams. Don't mess it up! Link here: https://www.amazon.com/Funny-Data-Systems-Business-Analyst/dp/B07FNW9FGJ/ref=sr_1_3?dchild=1&keywords=data+analyst+tshirt&qid=1626655184&sr=8-3"
   
    msg = f"Topico: {subject}\n\n{body}"
    
    server.sendmail(
        'msvf96@sa.com',
        msg
     
    )

##### Como se construyó la funcion de arriba

In [55]:

price = soup2.find("span", class_='a-offscreen').text
print(price)
price = price.strip()

print(price)

### los characetrs del price cambian dependiendo del precio del producto, lo ideal sería extraer 
# caracteres con la funcion de abajo
### Pero en caso sea algo sencillo puedes usar los indices para quedarte solo con los numeros que quieras
price = price[:]

## Tienes que eliminar las comas de los precios que tengan mas o igual de 1000 xq hay una COMA

coma=[",","S","$","U"]
res=[j for i,j in enumerate(price) if j not in coma]
price = "".join(res)

#price = price.strip(",")     

price = float(price)


                         $15.99
                        
$15.99


In [56]:
price

15.99

In [54]:
URL = "https://www.amazon.com/JDCUBAO-Lionel-Messi-dormitorio-decorativas/dp/B0BR3BM36M/ref=sr_1_2_sspa?__mk_es_US=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=2BGCYOGCOFVLN&keywords=messi&qid=1676251356&sprefix=messi%2Caps%2C383&sr=8-2-spons&psc=1&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUEyRzVBRzFaSjBXUE8yJmVuY3J5cHRlZElkPUEwNDY5OTIxMk9UMkVDSE40M09QSiZlbmNyeXB0ZWRBZElkPUExMDA2OTYwTkM1ODkxWEFYTjlUJndpZGdldE5hbWU9c3BfYXRmJmFjdGlvbj1jbGlja1JlZGlyZWN0JmRvTm90TG9nQ2xpY2s9dHJ1ZQ=="
#URL = "https://www.amazon.com/-/es/GeForce-256-Bit-Tri-Frozr-Lovelace-Architecture/dp/B0BL668N1X/ref=sr_1_1?__mk_es_US=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=3PWX4RL6HZ7DG&keywords=RTX+4080&qid=1676251689&sprefix=rtx+408%2Caps%2C173&sr=8-1"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

page = requests.get(URL, headers=headers)

soup = BeautifulSoup(page.content, "html.parser")

soup2 = BeautifulSoup(soup.prettify(), "html.parser")
